<a href="https://colab.research.google.com/github/cnndabbler/agentic-workflows/blob/main/run_pydantic_ai_with_Ollama_and_DeepSeek_R1_Step_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pydantic Agents running DeepSeek-R1 using Ollama

In [1]:
!pip install -q pydantic_ai
!pip install -q pydantic
!pip install -q rich

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.3/95.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.2/223.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.9/252.9 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.8/210.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.6/271.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 81.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.38.0 which is incompatible.


# Initiate Ollama and load the models in the GPU

In [2]:
import subprocess
import time

# Run the Ollama installation script
subprocess.run("curl -fsSL https://ollama.com/install.sh | sh", shell=True, check=True)

# Start the Ollama server in the background
ollama_process = subprocess.Popen(["ollama", "serve"])

# Allow some time for the server to initialize
time.sleep(5)

# Reasoner model on L4 GPU
# !ollama pull deepseek-r1:14b > /dev/null 2>&1
# !ollama pull deepseek-r1:14b

# Reasoner model on A100 GPU
!ollama pull deepseek-r1:32b > /dev/null 2>&1
# !ollama pull deepseek-r1:32b

# Parser model
!ollama pull qwen2.5 > /dev/null 2>&1
# !ollama pull qwen2.5

!ollama list

NAME               ID              SIZE      MODIFIED               
qwen2.5:latest     845dbda0ea48    4.7 GB    Less than a second ago    
deepseek-r1:32b    38056bbcbb2d    19 GB     38 seconds ago            


# Check the two models have been loaded in the GPU

In [56]:
!ollama list

NAME               ID              SIZE      MODIFIED       
qwen2.5:latest     845dbda0ea48    4.7 GB    30 minutes ago    
deepseek-r1:32b    38056bbcbb2d    19 GB     31 minutes ago    


# Setup Ollama Models for Agentic AI

In [4]:
from typing import List
from pydantic import BaseModel, Field, confloat
import asyncio
from pydantic_ai import Agent, RunContext
from rich.console import Console
from rich.table import Table
from rich import box

In [31]:
# Initialize models

from pydantic_ai.models.openai import OpenAIModel

# DeepSeekR1
deepseek_reasoner_model = OpenAIModel(
    'deepseek-r1:32b',
    # 'deepseek-r1:14b',
    base_url="http://localhost:11434/v1",
    api_key="whatever",
)

deepseek_parser_model = OpenAIModel(
    'qwen2.5',
    base_url="http://localhost:11434/v1",
    api_key="whatever",
)

# Pydantic Classes

In [6]:
from typing import List
from pydantic import BaseModel, Field, confloat
import asyncio
from pydantic_ai import Agent, RunContext
from rich.console import Console
from rich.table import Table
from rich import box

# Define the Joke model
class Joke(BaseModel):
    setup: str = Field(..., description="The setup part of the joke")
    punchline: str = Field(..., description="The punchline of the joke")
    category: str = Field(..., description="Category of the joke (e.g., 'pun', 'wordplay', 'dad joke')")
    thinking: str = Field(..., description="the thought process and planning without change")


In [51]:
# Create Joke Creator Agent
joke_creator = Agent(
    model=deepseek_reasoner_model,
    result_type=str,
    deps_type=str,  # Takes a string prompt as input
    system_prompt=(
        "You are a joke creator that specializes in creating funny, family-friendly dad jokes. "
        "Your output MUST follow this EXACT format with EXACT field names:\n\n"
        "Thinking: \n<think>\n[your entire thought process without changes]\n</think>\n\n"
        "Setup: [setup text]\n"
        "Punchline: [punchline text]\n"
        "Category: [category]\n"
        # "Thinking: [brief explanation]\n"
    ),
)

# Binding to Pydantic Joke class: Failure

- You will see the following error:
- BadRequestError: Error code: 400 - {'error': {'message': 'registry.ollama.ai/library/deepseek-r1:32b does not support tools', 'type': 'api_error', 'param': None, 'code': None}}

In [57]:
# Create Joke Creator Agent binding result_type to Joke class
joke_creator_with_pydantic_class = Agent(
    model=deepseek_reasoner_model,
    result_type=Joke,
    deps_type=str,  # Takes a string prompt as input
    system_prompt=(
        "You are a joke creator that specializes in creating funny, family-friendly dad jokes. "
        "Your output MUST follow this EXACT format with EXACT field names:\n\n"
        "Thinking: \n<think>\n[your detailed thought process between the tags without any changes]\n</think>\n\n"
        "Setup: [setup text]\n"
        "Punchline: [punchline text]\n"
        "Category: [category]\n"
    ),
)

NOTE: Only run the cell below to verify that Pydantic Agent does not support DeepSeek-R1

In [9]:
# joke_text = await joke_creator_with_pydantic_class.run("Create a dad joke")
# print("\nRaw joke text:")
# print("-" * 50)
# print(joke_text.data)
# print("-" * 50)

# Using a Parser Agent to successfully bind to Pydantic Class

In [58]:
# Create Parser Agent with 3 retries max
joke_parser = Agent(
    model=deepseek_parser_model,
    result_type=Joke,
    deps_type=str,  # Takes raw text as input
    retries=5,
    system_prompt=(
        "You are a joke parser that extracts components from the input text to create a Joke object. "
        "Your ONLY task is to find and extract these components from the input text without making any changes:\n"
        "1. The setup (after 'Setup:')\n"
        "2. The punchline (after 'Punchline:')\n"
        "3. The category (after 'Category:')\n"
        "4. The thinking (exact text tags without changes between <think> tags)\n\n"
        "Return the components in this format:\n"
        "setup=[text after Setup:]\n"
        "punchline=[text after Punchline:]\n"
        "category=[text after Category:]\n"
        "thinking: \n<think>\n[your detailed thought process between the tags without any changes]\n</think>\n\n"
        "DO NOT generate new content or modify the text in any way.\n"
        "DO NOT include any other text in your response."
    ),
)



In [59]:
# Step 1: Create raw joke text using DeepSeek-R1
print("Creating joke...")
joke_text = await joke_creator.run(
    "Create a dad joke",
    model_settings={"max_tokens": 1000}
    )
if not joke_text:
    raise ValueError("Failed to create joke")
print("\nRaw joke text:")
print("-" * 50)
print(joke_text.data)
print("-" * 50)

Creating joke...

Raw joke text:
--------------------------------------------------
<think>
Okay, so I need to create a dad joke using that specific format. The user provided an example where they used "octopus" and "tentacles" as wordplay, categorized it under marine life.

Hmm, I should think of another type that's family-friendly and has a pun or play on words. Maybe something involving animals since that's always fun. Let me brainstorm...

What about something with horses? I remember there are jokes like why did the horse jump in the bathtub? Because it heard it rained on plains. Wait, that might be more of a kid joke. Or maybe using "neighth" as a play on "night." But how can I make that into a dad joke?

Alternatively, what about cows? Cows moos and maybe something related to milk. Like why did the cow cross the road? To get to the udder side! Oh, that's a classic one.

Wait, let me see... Maybe something with elephants, like their trunks or how they use them. Or perhaps birds an

In [60]:
# Step 2: Parse the text into a Pydantic model using Qwen2.5
print("\nParsing joke...")
parsed_joke = await joke_parser.run(
    joke_text.data)
if not parsed_joke:
    raise ValueError("Failed to parse joke")
print("\nParsed joke text:")
print("-" * 50)
print(parsed_joke.data)
print("-" * 50)



Parsing joke...

Parsed joke text:
--------------------------------------------------
setup='Why did the chicken go to school?' punchline='To get egg-sellent grades!' category='Farm Animals/Birds' thinking="<think>I started with a classic setup of why did the chicken go to school since it's familiar and can resonate well. The punchline 'To get egg-sellent grades!' uses wordplay on 'excellent' with 'egg' which keeps it clean and family-friendly, fitting the dad joke category well.</think>"
--------------------------------------------------


In [61]:
# Step 3: Display results
console = Console()

# Create assessment table
table = Table(title="Joke Parsed", box=box.ROUNDED)
table.add_column("Property", style="cyan", width=20)
table.add_column("Value", style="yellow")

# Add joke content
table.add_row("Setup", parsed_joke.data.setup)
table.add_row("Punchline", parsed_joke.data.punchline)
table.add_row("Category", parsed_joke.data.category)
table.add_row("","")


# Add thinking process
table.add_row("", "")
table.add_row("[bold]Thinking Process", "")
thinking_lines = parsed_joke.data.thinking.split('\n')
for line in thinking_lines:
    if line.strip():
        table.add_row("", line)

# Print the table
console.print("\n")
console.print(table)

                                                    Joke Parsed                                                    
╭──────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────╮
│ Property             │ Value                                                                                    │
├──────────────────────┼──────────────────────────────────────────────────────────────────────────────────────────┤
│ Setup                │ Why did the chicken go to school?                                                        │
│ Punchline            │ To get egg-sellent grades!                                                               │
│ Category             │ Farm Animals/Birds                                                                       │
│                      │                                                                                          │
│                      │                                                                                          │
│ Thinking Process     │                                                                                          │
│                      │ <think>I started with a classic setup of why did the chicken go to school since it's     │
│                      │ familiar and can resonate well. The punchline 'To get egg-sellent grades!' uses wordplay │
│                      │ on 'excellent' with 'egg' which keeps it clean and family-friendly, fitting the dad joke │
│                      │ category well.</think>                                                                   │
╰──────────────────────┴──────────────────────────────────────────────────────────────────────────────────────────╯